In [1]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

import pickle
import torch
import importlib
import numpy as np
import pandas as pd
import copy

import src.bin.tensorify as tensorify 
import src.utils.data_conversion_utils as conversions
import src.data_manager.student_life_var_binned_data_manager as data_manager
import src.bin.trainer as trainer

from sklearn import metrics

from torch import nn
from copy import copy
from copy import deepcopy
from src import definitions
from src.bin import validations
from src.bin import statistics
from src.bin import plotting
from src.utils.read_utils import read_pickle
from src.utils import student_utils
from src.data_manager import sub_sampler
from src.data_manager import cross_val
from src.data_manager import helper as data_manager_helper
from tabulate import tabulate
from src.models import autoencoder
from src.models import multitask_learning
from src.models import user_dense_heads

importlib.reload(validations)
importlib.reload(autoencoder)
importlib.reload(multitask_learning)
importlib.reload(user_dense_heads)
importlib.reload(sub_sampler)
importlib.reload(statistics)
importlib.reload(cross_val)
importlib.reload(definitions)
importlib.reload(conversions)
importlib.reload(tensorify)
importlib.reload(plotting)
importlib.reload(trainer)
importlib.reload(data_manager)
importlib.reload(student_utils)
importlib.reload(data_manager_helper)

pd.set_option('max_rows', 10000)
feature_list = data_manager.FEATURE_LIST

In [2]:
# ##### Pickle #####
# data = read_pickle('../data/training_data/shuffled_splits/training_date_normalized_shuffled_splits_select_features.pkl')
data = read_pickle('../data/training_data/shuffled_splits/training_date_normalized_shuffled_splits_select_features_no_prev_stress.pkl')


#### Data ####
# student_list = [53, 46, 7, 49, 22, 24, 2]
# student_list = [ 7]
# data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=True, 
#                                                          fill_na=True, flatten_sequence=False)

# ###### Randomize ######
# train_ids, val_ids, test_ids = cross_val.random_stratified_splits(data)
# data['train_ids'] = train_ids
# data['val_ids'] = val_ids
# data['test_ids'] = test_ids

# data = sub_sampler.get_sub_sampled_sequences(data)

############# Stats ############# 
# unnormalized_data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, fill_na=False)
# statistics_df, raw_df = statistics.get_statistics_on_data_dict(unnormalized_data, feature_list)
# print(statistics.get_train_test_val_label_counts_from_raw_data(data))


# #### Data ####
# student_list = [53, 46, 7, 49, 22, 24, 2]
# student_list = [2,  46, 7]
# # student_list = [ 7]
# data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, 
#                                                          fill_na=True, flatten_sequence=False)

# ###### Randomize ######
# train_ids, val_ids, test_ids = cross_val.random_stratified_splits(data, stratify_by='students')
# data['train_ids'] = train_ids
# data['val_ids'] = val_ids
# data['test_ids'] = test_ids



# data = sub_sampler.get_sub_sampled_sequences(data)

############ Stats ############# 
# unnormalized_data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, fill_na=False)
# statistics_df, raw_df = statistics.get_statistics_on_data_dict(unnormalized_data, feature_list)
print(statistics.get_train_test_val_label_counts_from_raw_data(data))

      Train    Val    Test
--  -------  -----  ------
 0      111     45      29
 1      123     55      38
 2       93     30      21


In [12]:
data_df = pd.DataFrame()

for key in data['data']:
    data_df = data_df.append(pd.DataFrame([data['data'][key][definitions.COVARIATE_DATA_IDX]]))

print(data_df)

     0    1     2    3
0  3.0  1.0  10.0  0.0
0  4.0  1.0   7.0  0.0
0  5.0  1.0   7.0  0.0
0  5.0  1.0   8.0  0.0
0  6.0  1.0   8.0  0.0
0  0.0  1.0   8.0  0.0
0  2.0  2.0   8.0  0.0
0  3.0  2.0   6.0  0.0
0  4.0  2.0   8.0  0.0
0  5.0  3.0   7.0  0.0
0  5.0  3.0   7.0  0.0
0  6.0  1.0   9.0  0.0
0  1.0  2.0   6.0  0.0
0  2.0  1.0   8.0  0.0
0  2.0  2.0   5.0  0.0
0  3.0  2.0   5.0  0.0
0  4.0  2.0   6.0  0.0
0  5.0  1.0   9.0  0.0
0  6.0  1.0   8.0  0.0
0  1.0  1.0   8.0  0.0
0  1.0  1.0   8.0  0.0
0  2.0  1.0   7.0  1.0
0  4.0  1.0   8.0  1.0
0  5.0  2.0   7.0  1.0
0  4.0  1.0   7.0  1.0
0  6.0  1.0   7.0  1.0
0  6.0  1.0   7.0  1.0
0  6.0  1.0   7.0  1.0
0  0.0  1.0   7.0  1.0
0  3.0  2.0   7.0  0.0
0  4.0  3.0   7.0  0.0
0  5.0  2.0   8.0  0.0
0  6.0  2.0   7.0  0.0
0  2.0  2.0   8.0  0.0
0  3.0  3.0   8.0  0.0
0  4.0  2.0   8.0  0.0
0  6.0  2.0   7.0  0.0
0  0.0  2.0   9.0  0.0
0  0.0  2.0   7.0  0.0
0  1.0  2.0   7.0  0.0
0  3.0  2.0   8.0  0.0
0  5.0  2.0   8.0  0.0
0  6.0  2.0

In [34]:
sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len = statistics.get_sequence_length_and_num_features_from_data(data)
print("sequence_len: {} num_features: {} histogram_seq_len: {} histogram_num_feats: {} covariate_len: {}".format(
        sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len))


sequence_len: 1440 num_features: 10 histogram_seq_len: 25 histogram_num_feats: 9 covariate_len: 4


In [31]:
sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len = statistics.get_sequence_length_and_num_features_from_data(data)
print("sequence_len: {} num_features: {} histogram_seq_len: {} histogram_num_feats: {} covariate_len: {}".format(
        sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len))

for idx, key in enumerate(data["data"]):

    actual_data, misssing_flags, time_delta, covariates, historgram, label = data['data'][key]
    
    del covariates[3]
    
    data['data'][key] = (actual_data, misssing_flags, time_delta, covariates, historgram, label)
    
    
sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len = statistics.get_sequence_length_and_num_features_from_data(data)
print("sequence_len: {} num_features: {} histogram_seq_len: {} histogram_num_feats: {} covariate_len: {}".format(
        sequence_len, num_features, histogram_seq_len, hist_num_feats, covariate_len))    
    
    
outfile = open('../data/training_data/shuffled_splits/training_date_normalized_shuffled_splits_select_features_no_prev_stress.pkl',
               'wb')
pickle.dump(data,outfile)
outfile.close()    
    
    
    

sequence_len: 1440 num_features: 10 histogram_seq_len: 25 histogram_num_feats: 9 covariate_len: 5
sequence_len: 1440 num_features: 10 histogram_seq_len: 25 histogram_num_feats: 9 covariate_len: 4
